# Image plates of duckweed in growth assay experiment

Ensure the following before running anything in this notebook:
 - The machine is switched on, connected to the computer and all axes have been homed. 
 - The camera tool is accessible and in position 1 on the Jubilee
 
 Once those steps are complete run the cells in this notebook in order

In [ ]:
#Importing python libraries
import matplotlib
matplotlib.use('TkAgg') 
from matplotlib import pyplot as plt
import numpy as np
import os
from pathlib import Path
import pandas as pd
import json
import threading
from datetime import date
import ipywidgets as widgets

# from utils.CameraUtils import *
# from utils.MachineUtils import *


In [ ]:
from science_jubilee.Machine import Machine
from science_jubilee.decks.Deck import Deck
from science_jubilee.tools.Camera import Camera

In [ ]:
# Initialize your connection to the machine
m = Machine()
deck = m.load_deck("lab_automation_deck_MA")

camera = Camera(3, "Camera")

m.load_tool(camera)

In [ ]:
# Load the plates used for the experiment
plate1 = deck.load_labware("greiner_24_wellplate_3300ul", 1)
plate2 = deck.load_labware("greiner_24_wellplate_3300ul", 2)
plate3 = deck.load_labware("greiner_24_wellplate_3300ul", 3)

## 1. Select config file and define directory for storing output image data

### Select config file
Select the config file that matches the experiment you are collecting data for

In [ ]:
root_dir = Path(os.path.abspath('')).resolve().parent
config_file_dir = os.path.join(root_dir, "experiments")

config_opts = os.listdir(config_file_dir)
expt_choice = widgets.Dropdown(options = config_opts)
display(expt_choice)

In [ ]:
print("You selected:")
print(expt_choice.value)
print("If this is not correct re-select before continuing")

In [ ]:
experimental_config_file = os.path.join(config_file_dir, expt_choice.value, f'{expt_choice.value}.json')
with open(experimental_config_file, 'r') as f:
    experiment_data = json.load(f)
plate_info = experiment_data['sample_info']
df = pd.DataFrame(plate_info)

# Directory to save images to
data_dir = os.path.join(config_file_dir, expt_choice.value, 'images')

if not os.path.exists(data_dir):
    os.mkdir(data_dir) 

## 2. Load plates into machine

In [ ]:
#Drop bed down and add plates

m.move_to(z=100)

print('Load your experimental plates into the appropriate positions on the deck')

## 3. Focus camera
Run the cells below. It will move the camera over the first well of the first plate and then pop up a live camera window. Manually adjust the focus to the desired point by twisting the housing of the camera lens. You will need to loosen the nut that holds the lens in place to be able to adjust the focus and then tighten it again once you are done. Then hit the escape key to close the window

In [ ]:
m.pickup_tool(camera)

In [ ]:
#Move camera over first well to be imaged
plate1_A1 = plate1["A1"]

m.move_to(x=plate1_A1.x, y=plate1_A1.y)

In [ ]:
#Open a live video window
m.move_to(z=30) # your focus height
camera.video_stream() # esc to exit

## 4. Capture and save images

In [ ]:
%matplotlib inline

from utils.GrowthAssayHelpers import image_plates

image_plates(m, camera, df, data_dir, expt_choice.value)


# show_image_grid(data_dir)

In [ ]:
m.move_to(z=100)
print("Please remove plates from machine then hit any key to continue")
input()